How to Maintain nestpy
--------------------------
Sophia Andaloro 
<br>
Born: July 6, 2020
<br>
Last edit: November 11, 2020

## 1. Versioning and updating with NEST

Run:
```
git clone git@github.com:NESTCollaboration/nestpy.git
```
Then go to website:
https://github.com/NESTCollaboration/nest/tags
To see which tag you want to bind to. In this case 'v2.1.2', so then you go to
`src/nestpy/__init__.py`
<br>
Then change the version:
<br>
`__nest_version__ = '2.1.2'`

Then you have to install nestpy into some environment: <br>
`pip freeze | grep nest # no nestpy exists ` <br>
`python setup.py install`
<br>
such that:
```
python -c "import nestpy;print(nestpy.__nest_version__)"
```
Returns the version you want. Now run `sync.sh` (can run one at a time)

## sync.sh File:

```
# This script is used for syncing to the NEST main repository
# to ensure that the bindings bind to the same version of NEST
export VERSION=`python -c "import nestpy;print(nestpy.__nest_version__)"`
echo $VERSION 
git clone https://github.com/NESTCollaboration/nest.git nest_source
cd nest_source
git fetch --all --tags --prune
git checkout tags/v${VERSION} -b test
cd ..
cd src/nestpy
for filename in *.{cpp,hh}; do
  export REPO_FILE=`find ../../nest_source/ -name ${filename}`
  if [ ! -z "$REPO_FILE" -a "$REPO_FILE" != " " ]; then
    cp $REPO_FILE $filename
  fi
done
cd ../..
#rm -Rf nest_source
#git commit -m "Sync with $VERSION" -a
```

**For now, we have to go to execNEST function and put in the nuisparam and free params**:
```
NRYieldsParam = {11., 1.1, 0.0480, -0.0533, 12.6, 0.3, 2., 0.3, 2., 0.5, 1., 1.};
NRERWidthsParam = {1.,1.,0.1,0.5,0.19,2.25, 0.0015, 0.0553, 0.205, 0.45, -0.2};
ERWeightParam = {0.23, 0.77, 2.95, -1.44, 1.0, 1.0, 0., 0.};
  ```

**Additionally, nestpy requires a different minimum cmake version than the C++ version, so we need to remove the use of gcem in nestpy**:
    
In NEST.hh, comment out `#include "gcem.hpp" `
And change the lines:
```
         static constexpr double two_PI = 2. * M_PI;
         static constexpr double sqrt2 = gcem::sqrt(2.);
         static constexpr double sqrt2_PI = gcem::sqrt( 2. * M_PI );
         static constexpr double inv_sqrt2_PI = 1./gcem::sqrt( 2. * M_PI );
```
To
```
         double two_PI = 2. * M_PI;
         double sqrt2 = sqrt(2.);
         double sqrt2_PI = sqrt( 2. * M_PI );
         double inv_sqrt2_PI = 1./sqrt( 2. * M_PI );
```


Additionally, do a similar thing to RandomGen.hh:
Comment out `#include "gcem.hpp"`
and Change the lines:
```
   static constexpr double   xoroshiro128plus64_min = static_cast<double>(xoroshiro128plus64::min());
   static constexpr double   xoroshiro128plus64_minmax = static_cast<double>(xoroshiro128plus64::max() -xoroshiro128plus64::min());

   static constexpr double two_PI = 2. * M_PI;
   static constexpr double four_minus_PI_div_2 = 0.5*(4. - M_PI);
   static constexpr double sqrt2 = gcem::sqrt(2.);
   static constexpr double sqrt2_PI = gcem::sqrt( 2. * M_PI );
   static constexpr double sqrt2_div_PI =  gcem::sqrt(2./M_PI);
   static constexpr double log2 = gcem::log(2.);
```
to 
```
   double   xoroshiro128plus64_min = static_cast<double>(xoroshiro128plus64::min());
   double   xoroshiro128plus64_minmax = static_cast<double>(xoroshiro128plus64::max() - xoroshiro128plus64::min());

   double two_PI = 2. * M_PI;
   double four_minus_PI_div_2 = 0.5*(4. - M_PI);
   double sqrt2 = sqrt(2.);
   double sqrt2_PI = sqrt( 2. * M_PI );
   double sqrt2_div_PI =  sqrt(2./M_PI);
   double log2 = log(2.);
```

So in both files, we've removed any use of gcem for handling constant expressions. 

## Edit the histories 

- Edit HISTORY with new versions
```
cd nestpy
nano HISTORY.md 
git commit -m "Edit HISTORY.md for v<version>" HISTORY.md
nano src/nestpy/__init__.py
# Change the version of nestpy here 
```
- Push changes
```
git push
bumpversion minor # or patch, or major, depending on versioning changes
git push --tags
```

Then you'll want to go into travisCI and see if all checks complete on the commit 

# Download custom nestpy

`pip install --user -U git+https://github.com/NESTCollaboration/nestpy.git` 